**Ronaldlee Ejalu**

**CSC 583**

**HW#3**

In [446]:
import csv
import re
import os
import tarfile
import numpy as np
import pandas as pd
import pickle
import nltk
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download('vader_lexicon')
!pip install afinn
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
pd.options.display.max_columns = None

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [447]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [448]:
%cd /gdrive/MyDrive/CSC583

/gdrive/MyDrive/CSC583


neg and pos diirectories' path

In [449]:
directoryPath_neg = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/neg'
directoryPath_pos = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/pos'


train and test directories' path

In [450]:
directoryPath_train_o = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/train'
directoryPath_test_o = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/test'

In [451]:
directoryPath_train_labels = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/train_labels'
directoryPath_test_labels = '/gdrive/MyDrive/CSC583/data/homework3/txt_sentoken/test_labels'

regular expressions for extracting train and test sets

In [452]:
wordPattern_pos = r'^cv[0-7]{1}'
wordPattern_neg = r'^cv[8-9]{1}'

helper functions

In [453]:
#train_groundTruth_L = []
#test_groundTruth_L = []
def deriveTrainAndTest(directory_for_pos_or_neg, wordPattern_pos, srcPathFiles, destFilePath, labels, groudTruth_L):
  """Function that uses a regular expression that assigns files that start with cv0 
  through cv7 to the training and files that start with cv8 and cv9 to the test set"""
  
  for file in directory_for_pos_or_neg:
    t = re.findall(wordPattern_pos, file)
    if len(t) > 0:
      srcPath = os.path.join(srcPathFiles, file)
      destPath = os.path.join(destFilePath, file)
      os.rename(srcPath, destPath)
      groudTruth_L.append(labels)         

In [454]:
def readPlainDoc(filename_s):
  """Function that returns a string in lower case after reading the contents of the file"""
  fileNamePath = "data/homework3/sentiment_lexicon/" + filename_s
  with open(fileNamePath, 'r',  encoding = 'ISO-8859-1') as sampleFile:
    fileContents = sampleFile.read()
  # normalize all words to lower case
  return fileContents.lower()

In [455]:
def dumpListToFile(listName, groundTruth_labels):
  fileName = '/gdrive/MyDrive/CSC583/data/homework3/data/' + groundTruth_labels + '.csv'
  with open(fileName, 'w') as file:
    for item in listName:
      file.write(str(item) +'\n')

In [456]:
def readCSVFiles(fileName):
  filePath = '/gdrive/MyDrive/CSC583/data/homework3/data/' + fileName
  fileItems = []
  with open(filePath, 'r') as file:
    csvFile = csv.reader(file)
    for line in csvFile:
      fileItems.append(line)
  return fileItems

In [457]:
def cntPosNeg_adjec(tokenizedText, lexicon_positive_words, cnt_pos_adjec):
  """Function that returns either the number of positive or negative adjectives
  """
  pos_text = nltk.pos_tag(tokenizedText)
  for p_item in pos_text:
    if p_item[1].startswith('J') and p_item[0] in lexicon_positive_words:
      cnt_pos_adjec += 1
  return cnt_pos_adjec

In [458]:
def readDirectoryFiles(directoryFiles_train, filepath):
  """Function that returns a list of documents after reading all files in the directory"""
  train_L_Files = directoryFiles_train
  train_text_L = []
  
  for file in train_L_Files:
    filePath = filepath + file
    f_c = open(filePath, "r")
    train_text_L.append(f_c.read())
  return train_text_L

In [459]:
df_reviews = pd.DataFrame()
def remove_blanks(pd_f = df_reviews):
  # remove blanks from review
  blanks = []  # start with an empty list
  
  for lb,rv in pd_f.itertuples():
    if type(rv)==str:
      if rv.isspace():
        blanks.append(lb)     
  pd_f.drop(blanks, inplace=True)
  return pd_f

In [460]:
def vaderPolarityAndIntensity(train_L_Files):
  """Reviews the polarity and intensity of emotions"""
  sid = SentimentIntensityAnalyzer()
  pd_f = pd.DataFrame(data=train_L_Files, columns=['reviews'])
  pd_f = remove_blanks(pd_f)
  pd_f['scores'] = pd_f['reviews'].apply(lambda reviews: sid.polarity_scores(reviews))
  pd_f['compound']  = pd_f['scores'].apply(lambda score_dict: score_dict['compound'])
  columns_to_drop = ['scores', 'reviews']
  pd_f.drop(columns_to_drop, axis=1, inplace=True)
  polarityAndInt_L = pd_f['compound'].values.tolist()
  return polarityAndInt_L

In [461]:
def textProcess(train_L_Files):
  """"Function that returns a list of tuples of features derived from the different documents"""
  doc_num = 1
  # instantiate an afinn object 
  afn = Afinn()
  features_L = [] # list to store the tuples of the document's features

  for review in train_L_Files:
    tokenizedText = nltk.word_tokenize(review)
    word_cnt_of_doc = len(tokenizedText)
    #compute the polarity (scores) of each review.
    popularityScore=afn.score(review)
    # derive POS
    pos_text = nltk.pos_tag(tokenizedText)
    cnt_pos_adjec = 0
    cnt_neg_adjec = 0

    cnt_pos_adverbs = 0
    cnt_neg_adverbs = 0
    
    # derive positive and negative adjectives
    cnt_pos_adjec = cntPosNeg_adjec(tokenizedText, lexicon_positive_words, cnt_pos_adjec)
    cnt_neg_adjec = cntPosNeg_adjec(tokenizedText, lexicon_negative_words, cnt_neg_adjec)

    # derive positive and negative adverbs
    cnt_pos_adverbs = cntPosNeg_adjverbs(tokenizedText, lexicon_positive_words, cnt_pos_adverbs)
    cnt_neg_adverbs = cntPosNeg_adjverbs(tokenizedText, lexicon_negative_words, cnt_neg_adverbs)

    count_pos = 0
    count_neg = 0
    ind = 'doc'
    ind = ind + str(doc_num)
    #print("ind : %s" %ind)
    doc_num += 1
    #ind_L.append(ind)
    cnt_no_flag = 0
    #excl_flag = 0
    # get all the first-second-pronoun-dictionary-keys
    allkeys = first_second_pronouns_dict.keys()
    cnt_first_second_pronouns = 0
    cnt_exclamation = 0
    for item in tokenizedText:
      # count the positive lexicons in the doc
      if item in lexicon_positive_words:
        count_pos += 1
      
      # count the negative lexicons in the doc
      if item in lexicon_negative_words:
        count_neg += 1
      # check if 'no' is in the doc
      if item == 'no':
        cnt_no_flag += 1
      # check if 1st and 2nd pronouns exist in the document
      if item in allkeys:
        cnt_first_second_pronouns +=1
    if cnt_no_flag > 0:
      no_flag = 1
    else:
      no_flag = 0

    if count_pos > count_neg:
      sentiment = "positive"
    else:
      sentiment = "negative"
    features_L.append((ind, count_pos, count_neg, no_flag, cnt_first_second_pronouns, np.log(word_cnt_of_doc), popularityScore,\
                       cnt_pos_adjec,cnt_neg_adjec, cnt_pos_adverbs, cnt_neg_adverbs, sentiment))
  return features_L

In [462]:
pd_f = pd.DataFrame()
def normalizeDataSet(pd_t=pd_f):
  """Function that returns a normalized data set"""
  df_train_min_max_norm = pd_t.copy()
  # extract the sentiment values and convert them into a list
  sentiment_var = df_train_min_max_norm['sentiment'].values.tolist()
  df_train_min_max_norm.drop('sentiment', axis=1, inplace=True)
  for attr in df_train_min_max_norm.columns:
    if (df_train_min_max_norm[attr] != 0).any():
      df_train_min_max_norm[attr] = (df_train_min_max_norm[attr] - df_train_min_max_norm[attr].min())/ \
        (df_train_min_max_norm[attr].max() - df_train_min_max_norm[attr].min())
  # add back the sentiment attribute values to the normalized data set.
  df_train_min_max_norm['sentiment'] = sentiment_var
  return df_train_min_max_norm

In [463]:
def cntPosNeg_adjverbs(tokenizedText, lexicon_positive_words, cnt_pos_adverbs):  
  """Function that returns either the number of positive or negative adverbs  """  
  pos_text = nltk.pos_tag(tokenizedText)  
  for p_item in pos_text:    
    if p_item[1] == 'RB'and p_item[0][-2:] == 'ly' and p_item[0] in lexicon_negative_words:      
      cnt_pos_adverbs += 1  
  return cnt_pos_adverbs

In [464]:
#uncomment this line to download the file
!wget -c http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz -P data/homework3

--2023-04-23 23:35:45--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘data/homework3/review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  4.97MB/s    in 0.6s    

2023-04-23 23:35:46 (4.97 MB/s) - ‘data/homework3/review_polarity.tar.gz’ saved [3127238/3127238]



In [465]:
# uncomment this code to extract the file
#open the file
file = tarfile.open("data/homework3/review_polarity.tar.gz")

# extract the file
file.extractall('data/homework3')
file.close()

In [466]:
# uncomment the lines below to create directories
%rm -rf data/homework3/txt_sentoken/train
%rm -rf data/homework3/txt_sentoken/test
%mkdir data/homework3/txt_sentoken/train
%mkdir data/homework3/txt_sentoken/test

In [467]:
len(os.listdir(directoryPath_test_o))

0

In [468]:
len(os.listdir(directoryPath_train_o))

0

In [469]:
directoryFiles_pos = os.listdir(directoryPath_pos)
directoryFiles_neg = os.listdir(directoryPath_neg)
print("The number of files in the pos directory are %d" %len(directoryFiles_pos))
print("The number of files in the neg directory are %d" %len(directoryFiles_neg))

The number of files in the pos directory are 1000
The number of files in the neg directory are 1000


In [470]:
# initialize empty lists 
train_groundTruth_L = []
test_groundTruth_L = []

In [471]:
deriveTrainAndTest(directoryFiles_pos, wordPattern_pos,directoryPath_pos,directoryPath_train_o, 1, train_groundTruth_L)
deriveTrainAndTest(directoryFiles_neg, wordPattern_pos, directoryPath_neg, directoryPath_train_o, 0, train_groundTruth_L)
deriveTrainAndTest(directoryFiles_pos, wordPattern_neg, directoryPath_pos, directoryPath_test_o, 1, test_groundTruth_L)
deriveTrainAndTest(directoryFiles_neg, wordPattern_neg, directoryPath_neg, directoryPath_test_o, 0, test_groundTruth_L)

In [472]:
len(train_groundTruth_L)

1600

In [473]:
len(test_groundTruth_L)

400

In [474]:
# write the groundTruth_training list to the file
dumpListToFile(train_groundTruth_L, 'train_groundTruth_L')

In [475]:
# write the groundTruth_test list to the file
dumpListToFile(test_groundTruth_L, 'test_groundTruth_L')

In [476]:
directoryFiles_train = os.listdir(directoryPath_train_o)
directoryFiles_test = os.listdir(directoryPath_test_o)
print("There are %d files in the train data set." %len(directoryFiles_train))
print("There are %d files in the test data set." %len(directoryFiles_test))


There are 1600 files in the train data set.
There are 400 files in the test data set.


Get a list of files from the training data set

In [477]:
train_filepath='data/homework3/txt_sentoken/train/'
train_L_Files  = readDirectoryFiles(directoryFiles_train, train_filepath)
print("%d files of reviews read from the training data set." %len(train_L_Files))

test_filepath='data/homework3/txt_sentoken/test/'
test_L_Files  = readDirectoryFiles(directoryFiles_test, test_filepath)
print("%d files of reviews read from the test data set." %len(test_L_Files))


1600 files of reviews read from the training data set.
400 files of reviews read from the test data set.


Read in Sentiment Lexicon which consists of positive and negative words.

In [478]:
lexicon_positive_words = readPlainDoc('positive-words.txt')
type(lexicon_positive_words)

str

In [479]:
lexicon_negative_words = readPlainDoc('negative-words.txt')

In [480]:
# create a dictionary of first and second pronouns
first_second_pronouns_dict = {
								'i': 'i', 'we': 'we', 'me':'me', 'us':'us','mine':'mine',\
                              'ours':'ours', 'myself':'myself', 'ourselves': 'ourselves',\
                              'my':'my', 'our':'our', 'you':'you', 'thou':'thou', 'ye':'ye',\
                              'yours':'yours', 'thine':'thine', 'yourself':'yourself',\
                              'yourselves':'yourselves', 'thyself':'thyself', 'your':'your'
                           }
first_second_pronouns_dict = {
                              'i': 'i', 'we': 'we', 'me':'me', 'us':'us','mine':'mine',\
                              'ours':'ours', 'myself':'myself', 'ourselves': 'ourselves',\
                              'my':'my', 'our':'our', 'you':'you', 'thou':'thou', 'ye':'ye',\
                              'yours':'yours', 'thine':'thine', 'yourself':'yourself',\
                              'yourselves':'yourselves', 'thyself':'thyself', 'your':'your'
                           }

In [481]:
# derive the polarity and intensity of the training and test data sets reviews
polarityAndInt_L_train = vaderPolarityAndIntensity(train_L_Files)
polarityAndInt_L_test = vaderPolarityAndIntensity(test_L_Files)

In [482]:
# process the list of training documents
train_features_L = textProcess(train_L_Files)
# process the list of the test document
test_features_L = textProcess(test_L_Files)

In [483]:
df_train = pd.DataFrame(data=train_features_L, columns=['docs', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x10', 'x11','sentiment'])
df_train.set_index('docs', inplace=True)
df_train.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,sentiment
docs,,,,,,,,,,,
doc1,504,560,1,19,6.734592,-16.0,21,24,2,2,negative
doc2,177,194,1,6,5.673323,5.0,8,9,0,0,negative
doc3,327,358,0,8,6.331502,15.0,21,18,3,3,negative
doc4,308,329,1,6,6.352629,-14.0,28,25,2,2,negative
doc5,507,563,1,4,6.751101,-22.0,12,21,3,3,negative


In [484]:
df_test = pd.DataFrame(data=test_features_L, columns=['docs', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x10', 'x11','sentiment'])
df_test.set_index('docs', inplace=True)
# shuffle the data frame
#df_test=df_test.sample(frac=1)
df_test.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,sentiment
docs,,,,,,,,,,,
doc1,377,441,1,12,6.539586,-34.0,20,15,1,1,negative
doc2,161,192,0,2,5.828946,3.0,14,20,3,3,negative
doc3,253,295,1,7,6.146329,-13.0,11,30,4,4,negative
doc4,582,632,0,9,6.974479,11.0,29,47,8,8,negative
doc5,283,310,1,2,6.192362,-13.0,13,16,1,1,negative


In [485]:
print(df_train.shape)
print(df_test.shape)

(1600, 11)
(400, 11)


In [486]:
df_train_min_max_norm = normalizeDataSet(df_train)
df_train_min_max_norm['x9'] = polarityAndInt_L_train
df_train_min_max_norm['groundTruth'] = train_groundTruth_L
df_train_min_max_norm.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,sentiment,x9,groundTruth
docs,,,,,,,,,,,,,
doc1,0.331757,0.340810,1.0,0.279412,0.764247,0.331081,0.212766,0.205607,0.105263,0.105263,negative,0.9924,1
doc2,0.110811,0.112773,1.0,0.088235,0.553262,0.402027,0.074468,0.065421,0.000000,0.000000,negative,0.9160,1
doc3,0.212162,0.214953,0.0,0.117647,0.684111,0.435811,0.212766,0.149533,0.157895,0.157895,negative,0.8100,1
doc4,0.199324,0.196885,1.0,0.088235,0.688311,0.337838,0.287234,0.214953,0.105263,0.105263,negative,-0.8834,1
doc5,0.333784,0.342679,1.0,0.058824,0.767529,0.310811,0.117021,0.177570,0.157895,0.157895,negative,0.8029,1


In [487]:
df_train_min_max_norm.groupby(['groundTruth'])['groundTruth'].count()

groundTruth
0    800
1    800
Name: groundTruth, dtype: int64

In [488]:
df_test_min_max_norm = normalizeDataSet(df_test)
df_test_min_max_norm['x9'] = polarityAndInt_L_test
df_test_min_max_norm['groundTruth'] = test_groundTruth_L
df_test_min_max_norm.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,sentiment,x9,groundTruth
docs,,,,,,,,,,,,,
doc1,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,0.052632,negative,-0.9948,1
doc2,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.157895,negative,0.9555,1
doc3,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,0.210526,negative,-0.9772,1
doc4,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.421053,negative,0.9924,1
doc5,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,0.052632,negative,-0.9897,1


In [489]:
df_test_min_max_norm.groupby(['groundTruth'])['groundTruth'].count()

groundTruth
0    200
1    200
Name: groundTruth, dtype: int64

In [490]:
def writeDataFrameToCsv(filename, pd_t=df_train):
  fname = '/gdrive/MyDrive/CSC583/data/homework3/data/' + filename
  pd_t.to_csv(fname,  encoding='utf-8', index=False)

In [491]:
# write the training data frame to the filesystem
writeDataFrameToCsv('corpus_train.csv', df_train_min_max_norm)

In [492]:
# write the testing data frame to the filesystem
writeDataFrameToCsv('corpus_test.csv', df_test_min_max_norm)

# Creating a baseline lexicon-based classifier

In [493]:
# read the ground truth labels
train_groundTruth_Labels = readCSVFiles('train_groundTruth_L.csv')
test_groundTruth_Labels = readCSVFiles('test_groundTruth_L.csv')

In [494]:
# convert the training ground truth labels into integers
train_groundTruth_Labels = np.array(train_groundTruth_Labels, dtype=int)
train_groundTruth_Labels = list(train_groundTruth_Labels.flatten())

# convert the testing ground truth labels into integers
test_groundTruth_Labels = np.array(test_groundTruth_Labels, dtype=int)
test_groundTruth_Labels = list(test_groundTruth_Labels.flatten())

In [495]:
train_groundTruth_Labels[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [496]:
# read the csv file and create the train and test data frame
filenames = ['corpus_train.csv', 'corpus_test.csv']
train_df = pd.read_csv('/gdrive/MyDrive/CSC583/data/homework3/data/'+filenames[0])
train_df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,sentiment,x9,groundTruth
0,0.331757,0.340810,1.0,0.279412,0.764247,0.331081,0.212766,0.205607,0.105263,0.105263,negative,0.9924,1
1,0.110811,0.112773,1.0,0.088235,0.553262,0.402027,0.074468,0.065421,0.000000,0.000000,negative,0.9160,1
2,0.212162,0.214953,0.0,0.117647,0.684111,0.435811,0.212766,0.149533,0.157895,0.157895,negative,0.8100,1
3,0.199324,0.196885,1.0,0.088235,0.688311,0.337838,0.287234,0.214953,0.105263,0.105263,negative,-0.8834,1
4,0.333784,0.342679,1.0,0.058824,0.767529,0.310811,0.117021,0.177570,0.157895,0.157895,negative,0.8029,1


In [497]:
# shuffle train_df
train_df=train_df.sample(frac=1)
train_df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,sentiment,x9,groundTruth
855,0.322297,0.311526,0.0,0.294118,0.756043,0.486486,0.308511,0.224299,0.000000,0.000000,negative,0.9955,0
1348,0.343919,0.352648,1.0,0.176471,0.780578,0.310811,0.351064,0.364486,0.473684,0.473684,negative,-0.9889,0
1593,0.318243,0.320872,1.0,0.161765,0.755302,0.293919,0.223404,0.196262,0.157895,0.157895,negative,-0.9970,0
2,0.212162,0.214953,0.0,0.117647,0.684111,0.435811,0.212766,0.149533,0.157895,0.157895,negative,0.8100,1
395,0.181757,0.184424,1.0,0.029412,0.662850,0.395270,0.170213,0.205607,0.210526,0.210526,negative,0.6938,1


In [498]:
test_df = pd.read_csv('/gdrive/MyDrive/CSC583/data/homework3/data/'+filenames[1])
test_df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,sentiment,x9,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,0.052632,negative,-0.9948,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.157895,negative,0.9555,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,0.210526,negative,-0.9772,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.421053,negative,0.9924,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,0.052632,negative,-0.9897,1


In [499]:
# By using str.replace() replace positive string with 1 and negative with 0
test_df['sentiment'] = test_df['sentiment'].str.replace('negative','0')
test_df['sentiment'] = test_df['sentiment'].str.replace('positive','1')
test_df['sentiment'] = test_df['sentiment'].astype(int)

In [500]:
test_df.groupby(['sentiment'])['sentiment'].count()

sentiment
0    395
1      5
Name: sentiment, dtype: int64

In [501]:
test_df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,sentiment,x9,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,0.052632,0,-0.9948,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.157895,0,0.9555,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,0.210526,0,-0.9772,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.421053,0,0.9924,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,0.052632,0,-0.9897,1


In [502]:
filter_TP = (test_df['sentiment'] == 1) & (test_df['groundTruth'] == 1)
TP = len(test_df[filter_TP])
  
filter_TN = (test_df['sentiment'] == 0) & (test_df['groundTruth'] == 0)
TN = len(test_df[filter_TN])
  
filter_FP = (test_df['sentiment'] == 0) & (test_df['groundTruth'] == 1)
FP = len(test_df[filter_FP])
  
filter_FN = (test_df['sentiment'] == 1) & (test_df['groundTruth'] == 0)
FN = len(test_df[filter_FN])

In [503]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)


Confusion Matrix:
[[195   5]
 [200   0]]


In [504]:
Precision = TP/(TP+FP)
Recall = TP/(TP + FN)
#F1_Score = (2 * Precision * Recall)/(Precision+Recall)
accuracy = (TP+TN)/(TP+TN+FP+FN)
print('The Precision on the test set is %.2f.' %Precision)
print('The Recall on the test set is %.2f.' %Recall)
#print('The F1 Score on the test set is %.2f' %F1_Score)
print('Accuracy on the test set is %.2f.'%accuracy)

The Precision on the test set is 0.00.
The Recall on the test set is 0.00.
Accuracy on the test set is 0.49.


In [505]:
# separate x_train from y_train
def deriveXandYdatasets(pd_f=train_df):
  x_train = pd_f.copy()
  y_train = x_train['groundTruth'].values.tolist()
  train_sentiment = x_train['sentiment'].values.tolist()
  columns_to_drop=['groundTruth', 'sentiment']
  x_train.drop(columns_to_drop, axis=1, inplace=True)
  y_train = np.array(y_train)
  return x_train.T, y_train, train_sentiment


In [506]:
x_train, y_train, train_sentiment = deriveXandYdatasets(train_df)
print('The shape of x_train_arr is %s.'%str(x_train.shape))
print('The shape of y_train_arr is %s.'%str(y_train.shape))

The shape of x_train_arr is (11, 1600).
The shape of y_train_arr is (1600,).


In [507]:
x_test, y_test, test_sentiment = deriveXandYdatasets(test_df)
print('The shape of x_test_arr is %s.'%str(x_test.shape))
print('The shape of y_test_arr is %s.'%str(y_test.shape))


The shape of x_test_arr is (11, 400).
The shape of y_test_arr is (400,).


In [508]:
# create a copy of the x_train and x_test
blation_x_train = x_train.copy()
blation_x_test = x_test.copy()


In [509]:
y_train.shape

(1600,)

In [510]:
y_test.shape

(400,)

# Creating a logistic regression classfier from scratch

In [511]:
blationStudy = {}

In [512]:
def initialize_Weights_and_bias(lenOfdataset):
  return np.full((lenOfdataset, 1), 0.0, dtype=float), 0 # weights and bias

In [513]:
def sigmoid(x):
  if (x < 0).any():
    expo_x = np.exp(x)
    return expo_x / (1 + expo_x) # when x is in the range(0, -1)
  else:
    return 1 / (1 + np.exp(-x)) # when x is in the range (0, 1)

In [514]:
def crossEntropyLoss(y_true, y_hat):
  """Function that returns the loss function"""
  
  y_true_labels = y_true.astype(float)
  y_hat_labels = y_hat

  return -1 * np.mean(y_true_labels * np.log(y_hat_labels) + (1 - y_true_labels) * np.log(1 - y_hat_labels))

In [515]:
def deriveTheCost(x_train, y_true, w, b):
  noOfdims = x_train.shape[1]
  z = np.dot(w.T, x_train) + b

  y_hat = sigmoid(z)
  
  loss=crossEntropyLoss(y_true, y_hat)
  return np.sum(loss)/noOfdims   # average of the gradient of the cost function

In [516]:
def backprop(w, b, x_train, y_true):
  """Function that returns a tuple representing the weights and bias """
  lengthOfDataSet = x_train.shape[1]

  z = np.dot(w.T, x_train) + b
  y_hat = sigmoid(z)

  nabla_weights = (1 / lengthOfDataSet) * np.dot(x_train, (y_hat - y_true).T)   # average weights
  nabla_bias =   (1 / lengthOfDataSet) * np.sum(y_hat - y_true) # average of the bias

  return nabla_weights, nabla_bias # return the weights, bias

In [517]:
def update_mini_batch(epochs, weights, bias, x_train, y_train, learningRate):

  for num in range(epochs):
    
    g_cost = deriveTheCost(x_train, y_train, weights, bias)
    n_w, n_b = backprop(weights, bias, x_train, y_train)

    # update the weights and bias
    weights = weights - learningRate * n_w
    bias = bias - learningRate * n_b

    if num % 100 == 0:
      print('gradient cost: %f after %d epochs.' %(g_cost, num))
  return weights, bias
    

In [518]:
def predictions(w, b, x_test):
  """Function that returns predictions where output 1 if sigmoid(W.X + b) > 0.5 and 0 if sigmoid(W.X + b) < 0.5"""
  noOfdims = x_test.shape[1]
  predictions = np.zeros((1, noOfdims))

  z = np.dot(w.T, x_test) + b
  sigmoid_z = sigmoid(z)

  for num in range(sigmoid_z.shape[1]):
    if sigmoid_z[0, num] <= 0.5:
      predictions[0, num] = 0
    else:
      predictions[0, num] = 1
  return predictions


In [519]:
def fit_logistic_regression(epochs, x_train, y_train, x_test, y_test, learningRate):
  noOfFeatures = x_train.shape[0]

  # initialize the weights and bias
  weights, bias = initialize_Weights_and_bias(noOfFeatures)

  weights, bias = update_mini_batch(epochs, weights, bias, x_train, y_train, 0.01)
  print('weights: %s' %str(weights))
  print('bias: %s' %str(bias))
  predict_test = predictions(weights, bias, x_test)

  test_accuracy = (1 - np.mean(np.abs((predict_test - y_test)))) * 100
  print('test accuracy is %f' %test_accuracy)

  return predict_test, test_accuracy

In [520]:
def tuneModelPar(x_train, x_test):
  learningRate_L = [0.5, 0.25, 0.025, 0.002, 0.001, 0.01]
  epochs_L = [100, 500, 1000, 1500, 2000]
  for epoch in epochs_L:
    for item in range(len(learningRate_L)):
      print('using epoch %d and learningRate: %.4f' %(epoch, learningRate_L[item]))
      fit_logistic_regression(epoch, x_train, y_train, x_test, y_test, learningRate_L[item])
      print('\n')


In [521]:
X_test = pd.DataFrame() # initialize an empty data frame
def deriveConfusionMatrixPars(pd_df_test=X_test):
  """compute the confusion matrix parameters"""
  filter_TP = (pd_df_test['groundTruth'] == 1) & (pd_df_test['predictions'] == 1)
  TP = len(pd_df_test[filter_TP])
  
  filter_TN = (pd_df_test['groundTruth'] == 0) & (pd_df_test['predictions'] == 0)
  TN = len(pd_df_test[filter_TN])
  
  filter_FP = (pd_df_test['groundTruth'] == 0) & (pd_df_test['predictions'] == 1)
  FP = len(pd_df_test[filter_FP])
  
  filter_FN = (pd_df_test['groundTruth'] == 1) & (pd_df_test['predictions'] == 0)
  FN = len(pd_df_test[filter_FN])

  return TP, TN, FP, FN

In [522]:
def computeAndReport_metrics(TP, TN, FP, FN):
  Precision = TP/(TP+FP)
  Recall = TP/(TP + FN)
  F1_Score = (2 * Precision * Recall)/(Precision+Recall)
  accuracy = (TP+TN)/(TP+TN+FP+FN)
  print('The Precision on the test set is %.2f.' %Precision)
  print('The Recall on the test set is %.2f.' %Recall)
  print('The F1 Score on the test set is %.2f' %F1_Score)
  print('Accuracy on the test set is %.2f.'%accuracy)

In [523]:
def deriveDataSetWithXcolumn(columnName, pd_f_train=blation_x_train, pd_f_test=blation_x_test):
  #train
  blation_x_train_C = pd_f_train.T
  blation_x_test_C = pd_f_test.T
  blation_x_train_x8 = blation_x_train_C.copy()
  blation_x_train_x8.drop(columnName, axis=1, inplace=True)
  # test
  blation_x_test_x8 = blation_x_test_C.copy()
  blation_x_test_x8.drop(columnName, axis=1, inplace=True)
  blation_x_test_x8 = blation_x_test_x8.T
  blation_x_train_x8 = blation_x_train_x8.T
  return blation_x_train_x8, blation_x_test_x8

In [524]:
tuneModelPar(x_train, x_test)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00724753]
 [-0.0055426 ]
 [ 0.02749833]
 [ 0.00320833]
 [-0.00067338]
 [-0.01820632]
 [-0.01514771]
 [-0.00254014]
 [ 0.00216279]
 [ 0.00216279]
 [-0.11958539]]
bias: 0.0054685023314140595
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00724753]
 [-0.0055426 ]
 [ 0.02749833]
 [ 0.00320833]
 [-0.00067338]
 [-0.01820632]
 [-0.01514771]
 [-0.00254014]
 [ 0.00216279]
 [ 0.00216279]
 [-0.11958539]]
bias: 0.0054685023314140595
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00724753]
 [-0.0055426 ]
 [ 0.02749833]
 [ 0.00320833]
 [-0.00067338]
 [-0.01820632]
 [-0.01514771]
 [-0.00254014]
 [ 0.00216279]
 [ 0.00216279]
 [-0.11958539]]
bias: 0.0054685023314140595
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000

In [525]:
predict_test, test_accuracy = fit_logistic_regression(2000, x_train, y_train, x_test, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000408 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000402 after 700 epochs.
gradient cost: 0.000401 after 800 epochs.
gradient cost: 0.000400 after 900 epochs.
gradient cost: 0.000399 after 1000 epochs.
gradient cost: 0.000399 after 1100 epochs.
gradient cost: 0.000398 after 1200 epochs.
gradient cost: 0.000398 after 1300 epochs.
gradient cost: 0.000397 after 1400 epochs.
gradient cost: 0.000397 after 1500 epochs.
gradient cost: 0.000397 after 1600 epochs.
gradient cost: 0.000396 after 1700 epochs.
gradient cost: 0.000396 after 1800 epochs.
gradient cost: 0.000396 after 1900 epochs.
weights: [[-0.1112237 ]
 [-0.08554158]
 [ 0.31621626]
 [ 0.07253269]
 [ 0.05527871]
 [-0.19234135]
 [-0.23149383]
 [-0.03521032]
 [ 0.04528116]
 [ 0.045

In [526]:
# add the test accuracy to the dictionary
blationStudy['None'] = test_accuracy

In [527]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

X_test = x_test.T
X_test['predictions'] = predict_test_flattened
X_test['groundTruth'] = y_test_L
X_test.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,x9,predictions,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.157895,0.9555,0,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.421053,0.9924,0,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,0.052632,-0.9897,1,1


In [528]:
TP, TN, FP, FN = deriveConfusionMatrixPars(X_test)

In [529]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 106]
 [ 35  94]]


We compute the other metrics from the above confusion matrix.

In [530]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# Ablation Study

# Excluding the attribute x8

In [531]:
blation_x_train_x8, blation_x_test_x8 = deriveDataSetWithXcolumn('x8', blation_x_train, blation_x_test)
blation_x_test_x8.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [532]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x8, blation_x_test_x8)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00727321]
 [-0.00556849]
 [ 0.02745165]
 [ 0.00319317]
 [-0.00072914]
 [-0.01823757]
 [-0.01517066]
 [ 0.00215071]
 [ 0.00215071]
 [-0.11961001]]
bias: 0.005396095517841449
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00727321]
 [-0.00556849]
 [ 0.02745165]
 [ 0.00319317]
 [-0.00072914]
 [-0.01823757]
 [-0.01517066]
 [ 0.00215071]
 [ 0.00215071]
 [-0.11961001]]
bias: 0.005396095517841449
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00727321]
 [-0.00556849]
 [ 0.02745165]
 [ 0.00319317]
 [-0.00072914]
 [-0.01823757]
 [-0.01517066]
 [ 0.00215071]
 [ 0.00215071]
 [-0.11961001]]
bias: 0.005396095517841449
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00727321]
 [-0

Best parameters are epoch of 1000 and the learning rate of 0.01

In [533]:
predict_test, test_accuracy = fit_logistic_regression(1000, blation_x_train_x8, y_train, blation_x_test_x8, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000408 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000402 after 700 epochs.
gradient cost: 0.000401 after 800 epochs.
gradient cost: 0.000400 after 900 epochs.
weights: [[-0.05257381]
 [-0.03838324]
 [ 0.22005516]
 [ 0.04010708]
 [ 0.03994096]
 [-0.10882141]
 [-0.11980089]
 [ 0.02631226]
 [ 0.02631226]
 [-0.58969919]]
bias: 0.11919383049284997
test accuracy is 64.750000


In [534]:
blationStudy['x8'] = test_accuracy

In [535]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

blation_x_test_x8 = blation_x_test_x8.T
blation_x_test_x8['predictions'] = predict_test_flattened
blation_x_test_x8['groundTruth'] = y_test_L
blation_x_test_x8.head()

,x1,x2,x3,x4,x5,x6,x7,x10,x11,x9,predictions,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.214286,0.052632,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.142857,0.157895,0.157895,0.9555,0,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.107143,0.210526,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.321429,0.421053,0.421053,0.9924,0,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.130952,0.052632,0.052632,-0.9897,1,1


In [536]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x8)

In [537]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 106]
 [ 35  94]]


In [538]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# Excluding the attribute x11

In [539]:
blation_x_train_x11, blation_x_test_x11 = deriveDataSetWithXcolumn('x11', blation_x_train, blation_x_test)
blation_x_test_x11.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [540]:
# tune the model to get the best 
tuneModelPar(blation_x_train_x11, blation_x_test_x11)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00723819]
 [-0.00553317]
 [ 0.02751546]
 [ 0.00321377]
 [-0.00065281]
 [-0.01819485]
 [-0.01513955]
 [-0.00253189]
 [ 0.0021695 ]
 [-0.11957658]]
bias: 0.0054953137782354925
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00723819]
 [-0.00553317]
 [ 0.02751546]
 [ 0.00321377]
 [-0.00065281]
 [-0.01819485]
 [-0.01513955]
 [-0.00253189]
 [ 0.0021695 ]
 [-0.11957658]]
bias: 0.0054953137782354925
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00723819]
 [-0.00553317]
 [ 0.02751546]
 [ 0.00321377]
 [-0.00065281]
 [-0.01819485]
 [-0.01513955]
 [-0.00253189]
 [ 0.0021695 ]
 [-0.11957658]]
bias: 0.0054953137782354925
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00723819]
 

The best parameters are 1000 epochs and the learning rate of 0.01

In [541]:
predict_test, test_accuracy = fit_logistic_regression(1000, blation_x_train_x11, y_train, blation_x_test_x11, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000408 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000402 after 700 epochs.
gradient cost: 0.000401 after 800 epochs.
gradient cost: 0.000400 after 900 epochs.
weights: [[-0.05164662]
 [-0.03744353]
 [ 0.22129227]
 [ 0.04063756]
 [ 0.04164935]
 [-0.10794885]
 [-0.11897305]
 [-0.01220153]
 [ 0.02705759]
 [-0.58937387]]
bias: 0.12124193709874896
test accuracy is 64.750000


In [542]:
blationStudy['x11'] = test_accuracy

In [543]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)
blation_x_test_x11 = blation_x_test_x11.T
blation_x_test_x11['predictions'] = predict_test_flattened
blation_x_test_x11['groundTruth'] = y_test_L
blation_x_test_x11.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x9,predictions,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.9555,0,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.9924,0,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,-0.9897,1,1


In [544]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x11)

In [545]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 106]
 [ 35  94]]


In [546]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# Excluding the attribute x10

In [547]:
blation_x_train_x10, blation_x_test_x10 = deriveDataSetWithXcolumn('x10', blation_x_train, blation_x_test)
blation_x_test_x10.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [548]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x10, blation_x_test_x10)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00723819]
 [-0.00553317]
 [ 0.02751546]
 [ 0.00321377]
 [-0.00065281]
 [-0.01819485]
 [-0.01513955]
 [-0.00253189]
 [ 0.0021695 ]
 [-0.11957658]]
bias: 0.0054953137782354925
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00723819]
 [-0.00553317]
 [ 0.02751546]
 [ 0.00321377]
 [-0.00065281]
 [-0.01819485]
 [-0.01513955]
 [-0.00253189]
 [ 0.0021695 ]
 [-0.11957658]]
bias: 0.0054953137782354925
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00723819]
 [-0.00553317]
 [ 0.02751546]
 [ 0.00321377]
 [-0.00065281]
 [-0.01819485]
 [-0.01513955]
 [-0.00253189]
 [ 0.0021695 ]
 [-0.11957658]]
bias: 0.0054953137782354925
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00723819]
 

using the best parameters: epochs 1000 and learning rate of 0.01

In [549]:
predict_test, test_accuracy = fit_logistic_regression(1000, blation_x_train_x10, y_train, blation_x_test_x10, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000408 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000402 after 700 epochs.
gradient cost: 0.000401 after 800 epochs.
gradient cost: 0.000400 after 900 epochs.
weights: [[-0.05164662]
 [-0.03744353]
 [ 0.22129227]
 [ 0.04063756]
 [ 0.04164935]
 [-0.10794885]
 [-0.11897305]
 [-0.01220153]
 [ 0.02705759]
 [-0.58937387]]
bias: 0.12124193709874896
test accuracy is 64.750000


In [550]:
blationStudy['x10'] = test_accuracy

In [551]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)
blation_x_test_x10 = blation_x_test_x10.T
blation_x_test_x10['predictions'] = predict_test_flattened
blation_x_test_x10['groundTruth'] = y_test_L
blation_x_test_x10.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x11,x9,predictions,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.9555,0,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.9924,0,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,-0.9897,1,1


In [552]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x10)

In [553]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 106]
 [ 35  94]]


In [554]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# excluding the attribute x9

In [555]:
blation_x_train_x9, blation_x_test_x9 = deriveDataSetWithXcolumn('x9', blation_x_train, blation_x_test)
blation_x_test_x9.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [556]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x9, blation_x_test_x9)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00866872]
 [-0.00691878]
 [ 0.02531315]
 [ 0.00240988]
 [-0.00417529]
 [-0.02109021]
 [-0.01662988]
 [-0.0036727 ]
 [ 0.00157795]
 [ 0.00157795]]
bias: 0.0006710210484326156
test accuracy is 61.250000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00866872]
 [-0.00691878]
 [ 0.02531315]
 [ 0.00240988]
 [-0.00417529]
 [-0.02109021]
 [-0.01662988]
 [-0.0036727 ]
 [ 0.00157795]
 [ 0.00157795]]
bias: 0.0006710210484326156
test accuracy is 61.250000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00866872]
 [-0.00691878]
 [ 0.02531315]
 [ 0.00240988]
 [-0.00417529]
 [-0.02109021]
 [-0.01662988]
 [-0.0036727 ]
 [ 0.00157795]
 [ 0.00157795]]
bias: 0.0006710210484326156
test accuracy is 61.250000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00866872]
 

Using the best parameters of 0.01 as the learning rate and epoch 2000 since that is where we get the highest accuracy with the test data set

In [557]:
predict_test, test_accuracy = fit_logistic_regression(2000, blation_x_train_x9, y_train, blation_x_test_x9, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000432 after 100 epochs.
gradient cost: 0.000431 after 200 epochs.
gradient cost: 0.000431 after 300 epochs.
gradient cost: 0.000430 after 400 epochs.
gradient cost: 0.000429 after 500 epochs.
gradient cost: 0.000428 after 600 epochs.
gradient cost: 0.000428 after 700 epochs.
gradient cost: 0.000427 after 800 epochs.
gradient cost: 0.000426 after 900 epochs.
gradient cost: 0.000426 after 1000 epochs.
gradient cost: 0.000425 after 1100 epochs.
gradient cost: 0.000424 after 1200 epochs.
gradient cost: 0.000424 after 1300 epochs.
gradient cost: 0.000423 after 1400 epochs.
gradient cost: 0.000423 after 1500 epochs.
gradient cost: 0.000422 after 1600 epochs.
gradient cost: 0.000422 after 1700 epochs.
gradient cost: 0.000421 after 1800 epochs.
gradient cost: 0.000421 after 1900 epochs.
weights: [[-0.14372964]
 [-0.1097763 ]
 [ 0.34139443]
 [ 0.05905646]
 [-0.02634832]
 [-0.37170754]
 [-0.29646764]
 [-0.04632691]
 [ 0.04319388]
 [ 0.043

In [558]:
blationStudy['x9']=test_accuracy

In [559]:
blationStudy

{'None': 64.75, 'x8': 64.75, 'x11': 64.75, 'x10': 64.75, 'x9': 60.5}

In [560]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

blation_x_test_x9 = blation_x_test_x9.T
blation_x_test_x9['predictions'] = predict_test_flattened
blation_x_test_x9['groundTruth'] = y_test_L
blation_x_test_x9.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x10,x11,predictions,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,0.052632,1,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.157895,0,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,0.210526,1,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.421053,0,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,0.052632,1,1


In [561]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x9)

In [562]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[131  89]
 [ 69 111]]


In [563]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.62.
The Recall on the test set is 0.56.
The F1 Score on the test set is 0.58
Accuracy on the test set is 0.60.


# Excluding the column x7

In [564]:
blation_x_train_x7, blation_x_test_x7 = deriveDataSetWithXcolumn('x7', blation_x_train, blation_x_test)
blation_x_test_x7.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [565]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x7, blation_x_test_x7)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00738697]
 [-0.00568226]
 [ 0.02725193]
 [ 0.00312637]
 [-0.00097468]
 [-0.01838281]
 [-0.00266402]
 [ 0.00209834]
 [ 0.00209834]
 [-0.11976054]]
bias: 0.005076928159019498
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00738697]
 [-0.00568226]
 [ 0.02725193]
 [ 0.00312637]
 [-0.00097468]
 [-0.01838281]
 [-0.00266402]
 [ 0.00209834]
 [ 0.00209834]
 [-0.11976054]]
bias: 0.005076928159019498
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00738697]
 [-0.00568226]
 [ 0.02725193]
 [ 0.00312637]
 [-0.00097468]
 [-0.01838281]
 [-0.00266402]
 [ 0.00209834]
 [ 0.00209834]
 [-0.11976054]]
bias: 0.005076928159019498
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00738697]
 [-0

I will be using epoch 2000 and the learning rate of 0.01 since that is where we have the lowest cost and highest test accuracy

In [566]:
predict_test, test_accuracy = fit_logistic_regression(2000, blation_x_train_x7, y_train, blation_x_test_x7, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000409 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000403 after 700 epochs.
gradient cost: 0.000402 after 800 epochs.
gradient cost: 0.000401 after 900 epochs.
gradient cost: 0.000400 after 1000 epochs.
gradient cost: 0.000400 after 1100 epochs.
gradient cost: 0.000399 after 1200 epochs.
gradient cost: 0.000399 after 1300 epochs.
gradient cost: 0.000399 after 1400 epochs.
gradient cost: 0.000398 after 1500 epochs.
gradient cost: 0.000398 after 1600 epochs.
gradient cost: 0.000398 after 1700 epochs.
gradient cost: 0.000398 after 1800 epochs.
gradient cost: 0.000398 after 1900 epochs.
weights: [[-0.1224354 ]
 [-0.09666642]
 [ 0.30658042]
 [ 0.06616621]
 [ 0.0384936 ]
 [-0.20198077]
 [-0.04572863]
 [ 0.04033573]
 [ 0.04033573]
 [-0.704

In [567]:
blationStudy['x7']=test_accuracy

In [568]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

blation_x_test_x7 = blation_x_test_x7.T
blation_x_test_x7['predictions'] = predict_test_flattened
blation_x_test_x7['groundTruth'] = y_test_L
blation_x_test_x7.head()

,x1,x2,x3,x4,x5,x6,x8,x10,x11,x9,predictions,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.258772,0.171053,0.052632,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.421053,0.236842,0.157895,0.157895,0.9555,0,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.350877,0.368421,0.210526,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.456140,0.592105,0.421053,0.421053,0.9924,0,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.350877,0.184211,0.052632,0.052632,-0.9897,1,1


In [569]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x7)

In [570]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 106]
 [ 35  94]]


In [571]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# Excluding the column x6

In [572]:
blation_x_train_x6, blation_x_test_x6 = deriveDataSetWithXcolumn('x6', blation_x_train, blation_x_test)
blation_x_test_x6.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [573]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x6, blation_x_test_x6)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00748875]
 [-0.00578409]
 [ 0.02702538]
 [ 0.00306831]
 [-0.00127906]
 [-0.01536539]
 [-0.00274858]
 [ 0.0020513 ]
 [ 0.0020513 ]
 [-0.12000587]]
bias: 0.0046364719221812165
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00748875]
 [-0.00578409]
 [ 0.02702538]
 [ 0.00306831]
 [-0.00127906]
 [-0.01536539]
 [-0.00274858]
 [ 0.0020513 ]
 [ 0.0020513 ]
 [-0.12000587]]
bias: 0.0046364719221812165
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00748875]
 [-0.00578409]
 [ 0.02702538]
 [ 0.00306831]
 [-0.00127906]
 [-0.01536539]
 [-0.00274858]
 [ 0.0020513 ]
 [ 0.0020513 ]
 [-0.12000587]]
bias: 0.0046364719221812165
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00748875]
 

Our best parameters are when epoch is equal to 1500 and the learning rate of 0.01 since this is where we have the lowest cost and the highest test accuracy

In [574]:
predict_test, test_accuracy = fit_logistic_regression(1500, blation_x_train_x6, y_train, blation_x_test_x6, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000409 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000403 after 700 epochs.
gradient cost: 0.000402 after 800 epochs.
gradient cost: 0.000401 after 900 epochs.
gradient cost: 0.000400 after 1000 epochs.
gradient cost: 0.000399 after 1100 epochs.
gradient cost: 0.000399 after 1200 epochs.
gradient cost: 0.000399 after 1300 epochs.
gradient cost: 0.000398 after 1400 epochs.
weights: [[-0.08781908]
 [-0.06767348]
 [ 0.26805787]
 [ 0.05344999]
 [ 0.03238525]
 [-0.18219509]
 [-0.02902748]
 [ 0.03347039]
 [ 0.03347039]
 [-0.67668065]]
bias: 0.1393972301853369
test accuracy is 64.750000


In [575]:
blationStudy['x6']=test_accuracy

In [576]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

blation_x_test_x6 = blation_x_test_x6.T
blation_x_test_x6['predictions'] = predict_test_flattened
blation_x_test_x6['groundTruth'] = y_test_L
blation_x_test_x6.head()

,x1,x2,x3,x4,x5,x7,x8,x10,x11,x9,predictions,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.615765,0.214286,0.171053,0.052632,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.0,0.029412,0.366014,0.142857,0.236842,0.157895,0.157895,0.9555,0,1
2,0.164247,0.180464,1.0,0.102941,0.477557,0.107143,0.368421,0.210526,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.0,0.132353,0.768606,0.321429,0.592105,0.421053,0.421053,0.9924,0,1
4,0.191470,0.192881,1.0,0.029412,0.493735,0.130952,0.184211,0.052632,0.052632,-0.9897,1,1


In [577]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x6)

In [578]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 106]
 [ 35  94]]


In [579]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# Excluding the column X5

In [580]:
blation_x_train_x5, blation_x_test_x5 = deriveDataSetWithXcolumn('x5', blation_x_train, blation_x_test)
blation_x_test_x5.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [581]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x5, blation_x_test_x5)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00728775]
 [-0.0055831 ]
 [ 0.02741847]
 [ 0.00318481]
 [-0.01826337]
 [-0.01518273]
 [-0.00257519]
 [ 0.00214391]
 [ 0.00214391]
 [-0.11963243]]
bias: 0.005334347073938293
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00728775]
 [-0.0055831 ]
 [ 0.02741847]
 [ 0.00318481]
 [-0.01826337]
 [-0.01518273]
 [-0.00257519]
 [ 0.00214391]
 [ 0.00214391]
 [-0.11963243]]
bias: 0.005334347073938293
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00728775]
 [-0.0055831 ]
 [ 0.02741847]
 [ 0.00318481]
 [-0.01826337]
 [-0.01518273]
 [-0.00257519]
 [ 0.00214391]
 [ 0.00214391]
 [-0.11963243]]
bias: 0.005334347073938293
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00728775]
 [-0

I wil use epoch of 1000 and learning rate of 0.01

In [582]:
predict_test, test_accuracy = fit_logistic_regression(1000, blation_x_train_x5, y_train, blation_x_test_x5, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000408 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000402 after 700 epochs.
gradient cost: 0.000401 after 800 epochs.
gradient cost: 0.000400 after 900 epochs.
weights: [[-0.04853653]
 [-0.0343073 ]
 [ 0.22684934]
 [ 0.04241936]
 [-0.10337528]
 [-0.11630349]
 [-0.00949405]
 [ 0.02820543]
 [ 0.02820543]
 [-0.58677769]]
bias: 0.13229519113322252
test accuracy is 64.750000


In [583]:
blationStudy['x5']=test_accuracy

In [584]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

blation_x_test_x5 = blation_x_test_x5.T
blation_x_test_x5['predictions'] = predict_test_flattened
blation_x_test_x5['groundTruth'] = y_test_L
blation_x_test_x5.head()

,x1,x2,x3,x4,x6,x7,x8,x10,x11,x9,predictions,groundTruth
0,0.276770,0.301325,1.0,0.176471,0.258772,0.214286,0.171053,0.052632,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.0,0.029412,0.421053,0.142857,0.236842,0.157895,0.157895,0.9555,0,1
2,0.164247,0.180464,1.0,0.102941,0.350877,0.107143,0.368421,0.210526,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.0,0.132353,0.456140,0.321429,0.592105,0.421053,0.421053,0.9924,0,1
4,0.191470,0.192881,1.0,0.029412,0.350877,0.130952,0.184211,0.052632,0.052632,-0.9897,1,1


In [585]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x5)

In [586]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 106]
 [ 35  94]]


In [587]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# EXcluding the column x4

In [588]:
blation_x_train_x4, blation_x_test_x4 = deriveDataSetWithXcolumn('x4', blation_x_train, blation_x_test)
blation_x_test_x4.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [589]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x4, blation_x_test_x4)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00722941]
 [-0.00552435]
 [ 0.02753277]
 [-0.00063494]
 [-0.01818473]
 [-0.01513216]
 [-0.00252462]
 [ 0.00217094]
 [ 0.00217094]
 [-0.11956738]]
bias: 0.005518347378760217
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00722941]
 [-0.00552435]
 [ 0.02753277]
 [-0.00063494]
 [-0.01818473]
 [-0.01513216]
 [-0.00252462]
 [ 0.00217094]
 [ 0.00217094]
 [-0.11956738]]
bias: 0.005518347378760217
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00722941]
 [-0.00552435]
 [ 0.02753277]
 [-0.00063494]
 [-0.01818473]
 [-0.01513216]
 [-0.00252462]
 [ 0.00217094]
 [ 0.00217094]
 [-0.11956738]]
bias: 0.005518347378760217
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00722941]
 [-0

Using the epoch of 1000 and learning rate of 0.01.

In [590]:
predict_test, test_accuracy = fit_logistic_regression(1000, blation_x_train_x4, y_train, blation_x_test_x4, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000408 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000402 after 700 epochs.
gradient cost: 0.000401 after 800 epochs.
gradient cost: 0.000400 after 900 epochs.
weights: [[-0.05113154]
 [-0.03692733]
 [ 0.22214801]
 [ 0.04244231]
 [-0.10753275]
 [-0.11856435]
 [-0.01180263]
 [ 0.0269396 ]
 [ 0.0269396 ]
 [-0.5890946 ]]
bias: 0.12216087408896095
test accuracy is 64.750000


In [591]:
blationStudy['x4']=test_accuracy

In [592]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

blation_x_test_x4 = blation_x_test_x4.T
blation_x_test_x4['predictions'] = predict_test_flattened
blation_x_test_x4['groundTruth'] = y_test_L
blation_x_test_x4.head()

,x1,x2,x3,x5,x6,x7,x8,x10,x11,x9,predictions,groundTruth
0,0.276770,0.301325,1.0,0.615765,0.258772,0.214286,0.171053,0.052632,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.0,0.366014,0.421053,0.142857,0.236842,0.157895,0.157895,0.9555,0,1
2,0.164247,0.180464,1.0,0.477557,0.350877,0.107143,0.368421,0.210526,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.0,0.768606,0.456140,0.321429,0.592105,0.421053,0.421053,0.9924,0,1
4,0.191470,0.192881,1.0,0.493735,0.350877,0.130952,0.184211,0.052632,0.052632,-0.9897,1,1


In [593]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x4)

In [594]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 106]
 [ 35  94]]


In [595]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# Excluding the column x3

In [596]:
blation_x_train_x3, blation_x_test_x3 = deriveDataSetWithXcolumn('x3', blation_x_train, blation_x_test)
blation_x_test_x3.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [597]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x3, blation_x_test_x3)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00672021]
 [-0.00501043]
 [ 0.00353539]
 [ 0.00057262]
 [-0.01751391]
 [-0.01470305]
 [-0.00208476]
 [ 0.00240654]
 [ 0.00240654]
 [-0.11911808]]
bias: 0.0071341372215224744
test accuracy is 64.250000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00672021]
 [-0.00501043]
 [ 0.00353539]
 [ 0.00057262]
 [-0.01751391]
 [-0.01470305]
 [-0.00208476]
 [ 0.00240654]
 [ 0.00240654]
 [-0.11911808]]
bias: 0.0071341372215224744
test accuracy is 64.250000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00672021]
 [-0.00501043]
 [ 0.00353539]
 [ 0.00057262]
 [-0.01751391]
 [-0.01470305]
 [-0.00208476]
 [ 0.00240654]
 [ 0.00240654]
 [-0.11911808]]
bias: 0.0071341372215224744
test accuracy is 64.250000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00672021]
 

Using the epoch of 1000 and the learning rate of 0.01

In [598]:
predict_test, test_accuracy = fit_logistic_regression(1000, blation_x_train_x3, y_train, blation_x_test_x3, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000424 after 100 epochs.
gradient cost: 0.000418 after 200 epochs.
gradient cost: 0.000413 after 300 epochs.
gradient cost: 0.000410 after 400 epochs.
gradient cost: 0.000408 after 500 epochs.
gradient cost: 0.000406 after 600 epochs.
gradient cost: 0.000405 after 700 epochs.
gradient cost: 0.000404 after 800 epochs.
gradient cost: 0.000403 after 900 epochs.
weights: [[-0.03441813]
 [-0.01996512]
 [ 0.05216242]
 [ 0.08027374]
 [-0.08851347]
 [-0.10547972]
 [ 0.00241787]
 [ 0.03441017]
 [ 0.03441017]
 [-0.58645745]]
bias: 0.17147063615047214
test accuracy is 64.250000


In [599]:
blationStudy['x3']=test_accuracy

In [600]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

blation_x_test_x3 = blation_x_test_x3.T
blation_x_test_x3['predictions'] = predict_test_flattened
blation_x_test_x3['groundTruth'] = y_test_L
blation_x_test_x3.head()

,x1,x2,x4,x5,x6,x7,x8,x10,x11,x9,predictions,groundTruth
0,0.276770,0.301325,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,0.052632,-0.9948,1,1
1,0.080762,0.095199,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.157895,0.9555,0,1
2,0.164247,0.180464,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,0.210526,-0.9772,1,1
3,0.462795,0.459437,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.421053,0.9924,0,1
4,0.191470,0.192881,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,0.052632,-0.9897,1,1


In [601]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x3)

In [602]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[164 107]
 [ 36  93]]


In [603]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.72.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.64.


# Excluding the column x2

In [604]:
blation_x_train_x2, blation_x_test_x2 = deriveDataSetWithXcolumn('x2', blation_x_train, blation_x_test)
blation_x_test_x2.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x1,0.276770,0.080762,0.164247,0.462795,0.191470,0.340290,0.165154,0.844828,0.180581,0.068966,0.292196,0.491833,0.224138,0.428312,0.170599,0.676044,0.299456,0.085299,0.415608,0.308530,0.414701,0.181488,0.298548,0.270417,0.209619,0.196007,0.460980,0.110708,0.118875,0.394737,0.054446,0.457350,0.590744,0.281307,0.613430,0.400181,0.206897,0.302178,0.417423,0.280399,0.141561,0.312160,0.361162,0.107078,0.361162,0.341198,0.389292,0.451906,0.271325,0.511797,0.216878,0.232305,0.339383,0.257713,0.084392,0.161525,0.146098,0.000000,0.435572,0.209619,0.269510,0.113430,0.397459,0.215064,0.331216,0.388385,0.275862,0.353902,0.064428,0.376588,0.165154,0.700544,0.305808,0.225953,0.531760,0.035390,0.383848,0.301270,0.350272,0.207804,0.157895,0.088022,0.350272,0.432849,0.249546,0.205989,0.282214,0.058984,0.421053,0.391107,0.345735,0.393829,0.388385,0.866606,0.309437,0.156080,0.254083,0.175136,0.260436,0.158802,0.222323,0.102541,0.368421,0.134301,0.057169,0.273140,0.472777,0.302178,0.523593,0.134301,0.300363,0.458258,0.564428,0.157895,0.165154,0.241379,0.250454,0.291289,0.244102,0.154265,0.099819,0.432849,0.147913,0.385662,0.236842,0.238657,0.804900,0.388385,0.686933,0.122505,0.189655,0.156987,0.185118,0.784936,0.441924,0.382940,0.272232,0.230490,0.240472,0.199637,0.021779,0.370236,0.343920,0.205989,0.298548,0.217786,0.333938,0.112523,0.305808,0.388385,0.773140,0.427405,0.344828,0.243194,0.304900,0.147005,0.594374,0.200544,0.231397,0.263158,0.265880,0.126134,0.286751,0.380218,0.269510,0.325771,0.379310,0.282214,0.402904,0.098911,0.383848,0.389292,0.389292,0.638838,0.177858,0.729583,0.232305,0.218693,0.340290,0.300363,0.333938,0.430127,0.193285,0.253176,0.214156,0.333938,0.243194,0.368421,0.385662,0.182396,0.112523,0.390200,0.487296,0.080762,0.415608,0.750454,0.578040,0.186933,0.274047,0.220508,0.392015,0.559891,0.500907,0.434664,0.397459,0.415608,0.325771,0.754991,0.372051,0.459165,0.466425,0.392922,0.584392,0.210526,0.114338,0.333031,0.348457,0.802178,0.424682,0.770417,0.236842,0.245917,0.304900,0.207804,0.154265,0.026316,0.276770,0.245009,0.314882,0.303085,0.301270,0.257713,0.305808,0.383848,0.330309,0.082577,0.571688,0.448276,0.166062,0.462795,0.274955,0.055354,0.263158,0.155172,0.443739,0.580762,0.578040,0.087114,0.228675,0.284029,0.193285,0.251361,0.258621,0.284936,0.388385,0.258621,0.348457,0.226860,0.450998,0.451906,0.441924,0.175136,0.843920,0.270417,0.154265,0.369328,0.374773,0.172414,0.535390,0.427405,0.438294,0.320327,0.619782,0.492740,0.372051,0.422868,0.118875,0.090744,0.35

In [605]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x2, blation_x_test_x2)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00730928]
 [ 0.02738549]
 [ 0.00317149]
 [-0.0008066 ]
 [-0.01828121]
 [-0.01520117]
 [-0.00259363]
 [ 0.00213428]
 [ 0.00213428]
 [-0.11964746]]
bias: 0.005295628748955414
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00730928]
 [ 0.02738549]
 [ 0.00317149]
 [-0.0008066 ]
 [-0.01828121]
 [-0.01520117]
 [-0.00259363]
 [ 0.00213428]
 [ 0.00213428]
 [-0.11964746]]
bias: 0.005295628748955414
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00730928]
 [ 0.02738549]
 [ 0.00317149]
 [-0.0008066 ]
 [-0.01828121]
 [-0.01520117]
 [-0.00259363]
 [ 0.00213428]
 [ 0.00213428]
 [-0.11964746]]
bias: 0.005295628748955414
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00730928]
 [ 0

using the epoch of 1000 and the learning rate of 0.01

In [606]:
predict_test, test_accuracy = fit_logistic_regression(1000, blation_x_train_x2, y_train, blation_x_test_x2, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000408 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000402 after 700 epochs.
gradient cost: 0.000401 after 800 epochs.
gradient cost: 0.000400 after 900 epochs.
weights: [[-0.05373412]
 [ 0.21848237]
 [ 0.03940675]
 [ 0.03789152]
 [-0.10988531]
 [-0.12075689]
 [-0.01401971]
 [ 0.02579987]
 [ 0.02579987]
 [-0.59024508]]
bias: 0.11676162898575564
test accuracy is 64.500000


In [607]:
blationStudy['x2']=test_accuracy

In [608]:
predict_test_T = predict_test.T
predict_test_T = predict_test_T.astype(int)
# convert the numpy array into a list
predict_test_L = list(predict_test_T)
y_test_L = list(y_test)
predict_test_flattened = predict_test_T.flatten()
predict_test_flattened = list(predict_test_flattened)

blation_x_test_x2 = blation_x_test_x2.T
blation_x_test_x2['predictions'] = predict_test_flattened
blation_x_test_x2['groundTruth'] = y_test_L
blation_x_test_x2.head()

,x1,x3,x4,x5,x6,x7,x8,x10,x11,x9,predictions,groundTruth
0,0.276770,1.0,0.176471,0.615765,0.258772,0.214286,0.171053,0.052632,0.052632,-0.9948,1,1
1,0.080762,0.0,0.029412,0.366014,0.421053,0.142857,0.236842,0.157895,0.157895,0.9555,0,1
2,0.164247,1.0,0.102941,0.477557,0.350877,0.107143,0.368421,0.210526,0.210526,-0.9772,1,1
3,0.462795,0.0,0.132353,0.768606,0.456140,0.321429,0.592105,0.421053,0.421053,0.9924,0,1
4,0.191470,1.0,0.029412,0.493735,0.350877,0.130952,0.184211,0.052632,0.052632,-0.9897,1,1


In [609]:
TP, TN, FP, FN = deriveConfusionMatrixPars(blation_x_test_x2)

In [610]:
confusionMatrix = np.array([[TN, FN], [FP, TP]])
print('Confusion Matrix:')
print(confusionMatrix)

Confusion Matrix:
[[165 107]
 [ 35  93]]


In [611]:
computeAndReport_metrics(TP, TN, FP, FN)

The Precision on the test set is 0.73.
The Recall on the test set is 0.47.
The F1 Score on the test set is 0.57
Accuracy on the test set is 0.65.


# Excluding the column x1

In [612]:
blation_x_train_x1, blation_x_test_x1 = deriveDataSetWithXcolumn('x1', blation_x_train, blation_x_test)
blation_x_test_x1.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399
x2,0.301325,0.095199,0.180464,0.459437,0.192881,0.339404,0.168874,0.863411,0.190397,0.071192,0.298013,0.489238,0.238411,0.444536,0.165563,0.671358,0.298841,0.081954,0.423013,0.322020,0.415563,0.200331,0.296358,0.260762,0.210265,0.206126,0.455298,0.128311,0.138245,0.420530,0.066225,0.454470,0.611755,0.311258,0.596026,0.410596,0.203642,0.302980,0.389901,0.270695,0.149834,0.330298,0.372517,0.113411,0.372517,0.311258,0.398179,0.451987,0.296358,0.547185,0.228477,0.289735,0.345199,0.270695,0.091060,0.163907,0.152318,0.000000,0.454470,0.219371,0.287252,0.109272,0.385762,0.226821,0.323675,0.394040,0.290563,0.349338,0.067881,0.379139,0.167219,0.723510,0.322848,0.225166,0.536424,0.029801,0.378311,0.289735,0.367550,0.197848,0.149834,0.107616,0.357616,0.443709,0.254967,0.206954,0.278146,0.062086,0.408940,0.389073,0.340232,0.394868,0.403146,0.920530,0.303808,0.148179,0.264073,0.190397,0.282285,0.164735,0.235099,0.105960,0.383278,0.130795,0.061258,0.258278,0.447848,0.294702,0.509934,0.146523,0.315397,0.451987,0.578642,0.158113,0.181291,0.252483,0.247517,0.293874,0.253311,0.163907,0.091887,0.442881,0.141556,0.384934,0.225166,0.239238,0.807119,0.394868,0.702815,0.133278,0.202815,0.153974,0.192053,0.789735,0.433775,0.423841,0.264901,0.242550,0.254139,0.198675,0.028974,0.360099,0.343543,0.225166,0.304636,0.229305,0.346854,0.108444,0.314570,0.408940,0.753311,0.427980,0.355132,0.261589,0.302152,0.168046,0.581126,0.198675,0.239238,0.271523,0.257450,0.133278,0.265728,0.400662,0.268212,0.335265,0.359272,0.277318,0.420530,0.097682,0.398179,0.403146,0.379139,0.625000,0.183775,0.736755,0.218543,0.215232,0.324503,0.311258,0.334437,0.424669,0.206126,0.259106,0.217715,0.339404,0.240894,0.377483,0.381623,0.192881,0.120861,0.391556,0.500828,0.075331,0.424669,0.777318,0.544702,0.186258,0.271523,0.228477,0.369205,0.572020,0.480132,0.408113,0.386589,0.389073,0.319536,0.724338,0.378311,0.440397,0.485927,0.395695,0.590232,0.209437,0.108444,0.358444,0.357616,0.764073,0.390728,0.750828,0.235099,0.249172,0.307119,0.207781,0.157285,0.021523,0.275662,0.254139,0.308775,0.296358,0.307947,0.265728,0.316225,0.359272,0.314570,0.082781,0.553808,0.424669,0.177152,0.459437,0.257450,0.059603,0.254967,0.142384,0.466060,0.581954,0.567053,0.085265,0.221854,0.293046,0.192881,0.265728,0.246689,0.274834,0.383278,0.267384,0.357616,0.213576,0.466887,0.443709,0.453642,0.183775,0.826987,0.270695,0.143212,0.344371,0.376656,0.174669,0.504139,0.422185,0.432947,0.317881,0.586921,0.481788,0.399834,0.416391,0.114238,0.076987,0.34

In [613]:
# tune the model to get the best parameters
tuneModelPar(blation_x_train_x1, blation_x_test_x1)

using epoch 100 and learningRate: 0.5000
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00562212]
 [ 0.02735429]
 [ 0.00316122]
 [-0.00084381]
 [-0.0183027 ]
 [-0.01521646]
 [-0.00260848]
 [ 0.00212642]
 [ 0.00212642]
 [-0.11966804]]
bias: 0.005247384182648532
test accuracy is 64.000000


using epoch 100 and learningRate: 0.2500
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00562212]
 [ 0.02735429]
 [ 0.00316122]
 [-0.00084381]
 [-0.0183027 ]
 [-0.01521646]
 [-0.00260848]
 [ 0.00212642]
 [ 0.00212642]
 [-0.11966804]]
bias: 0.005247384182648532
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0250
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00562212]
 [ 0.02735429]
 [ 0.00316122]
 [-0.00084381]
 [-0.0183027 ]
 [-0.01521646]
 [-0.00260848]
 [ 0.00212642]
 [ 0.00212642]
 [-0.11966804]]
bias: 0.005247384182648532
test accuracy is 64.000000


using epoch 100 and learningRate: 0.0020
gradient cost: 0.000433 after 0 epochs.
weights: [[-0.00562212]
 [ 0

using the epoch of 1000 and the learning rate of 0.01

In [614]:
predict_test, test_accuracy = fit_logistic_regression(1000, blation_x_train_x1, y_train, blation_x_test_x1, y_test, 0.01)

gradient cost: 0.000433 after 0 epochs.
gradient cost: 0.000423 after 100 epochs.
gradient cost: 0.000417 after 200 epochs.
gradient cost: 0.000412 after 300 epochs.
gradient cost: 0.000408 after 400 epochs.
gradient cost: 0.000406 after 500 epochs.
gradient cost: 0.000404 after 600 epochs.
gradient cost: 0.000402 after 700 epochs.
gradient cost: 0.000401 after 800 epochs.
gradient cost: 0.000400 after 900 epochs.
weights: [[-0.04015785]
 [ 0.21769614]
 [ 0.03904768]
 [ 0.03683248]
 [-0.11046374]
 [-0.12128933]
 [-0.01453109]
 [ 0.02553687]
 [ 0.02553687]
 [-0.59063594]]
bias: 0.11551251779587841
test accuracy is 64.500000


In [615]:
blationStudy['x1']=test_accuracy

In [616]:
# create data frame
blationStudy_df = pd.DataFrame(list(blationStudy.items()), columns=['Excluded Feautures', 'Test Accuracy'])
blationStudy_df.head(11)

,Excluded Feautures,Test Accuracy
0,None,64.75
1,x8,64.75
2,x11,64.75
3,x10,64.75
4,x9,60.50
5,x7,64.75
6,x6,64.75
7,x5,64.75
8,x4,64.75
9,x3,64.25
